In [5]:
import numpy as np

import reactord as rd

In [33]:
def r_rate1(c, t, cons):
    return np.full(np.size(t), cons["k1"])

def r_rate2(c, t, cons):
    return np.full(np.size(t), cons["k2"])

In [34]:
  # Problem data
fa_in = 5 / 3600  # mol/s
f_volumetric = 10 * 0.001 / 60  # m3/s

k1 = 0.05 / 3600 / 0.001  # mol/s/m3
k2 = 0.15 / 3600 / 0.001  # mol/s/m3

v_pfr = 99 * 0.001  # m3

We create some substances first:

In [35]:
a = rd.Substance(name="A")
b = rd.Substance(name="B")
c = rd.Substance(name="C")
d = rd.Substance(name="D")

After that, the mixture of the substances is created:

In [36]:
mix = rd.mix.IdealSolution([a, b, c, d])

And the kinetic object is instantiated. The required parameters include the mixture object and a dictionary that contains the reactions in the following format:

reactions={"reaction_1": {**"eq"**: a > b, **"rate"**: r_rate1}, 
           "reaction_2":{**"eq"**: x > y, **"rate"**: r_rate2}...}

The keywords **"eq"** (for equation) and **"rate"** (for reaction rate) are mandatory

On the other hand, the dictionary kinetic_constants contains the rate constants
for the reactions. The keywords for the rate constants can be chosen freely.

In [37]:
kinetic = rd.Kinetic(
    mix=mix,
    reactions={"r1": {"eq": a > b, "rate": r_rate1},
               "r2": {"eq": a + b > (c + 2*d), "rate": r_rate2}},
    kinetic_constants={"k1": k1, "k2": k2},
)

The format "xa + yb > wc + zd" allows us to visually represent the reactions in an aesthetically pleasing Latex format. For that, we invoke the **irepr** property.

In [38]:
kinetic.irepr

<IPython.core.display.Math object>

<IPython.core.display.Math object>

The dunder method __len__ was overwritten to provide the number of reactions
in the class Kinetic.

Additionally, the dunder method __repr__ return the subtances contained in the mixture.

***PREGUNTAR SI ESTA BIEN, PQ APARECE TAMBIEN UNA REPRESENTACION DEL IREPR AHI***

In [46]:
len(kinetic)
print(kinetic.__repr__())

Mixture's substances: 
  * A 
  * B 
  * C 
  * D 

System's reactions: 
r1: A \rightarrow B 
r2: A + B \rightarrow C + 2 D 

